In [2]:
# !git clone https://github.com/pranjali97/RoBERTa_QA.git
# !unzip covid-qa.zip
# !pip install transformers

In [ ]:
import pandas as pd
import json
import ast

PART -1

In [3]:
def extract_data_fromJSON(filename):
    data = json.load(open(filename))
    extracted_data = []

    for row in data["data"]:
        doc = row["paragraphs"][0]
        for question in doc["qas"]:
            ele = {"context": doc["context"], "document_id": doc["document_id"]}
            ele["question"] = question["question"]
            ele["answers"] = {
                "text": [ans["text"] for ans in question["answers"]],
                "answer_start": [ans["answer_start"] for ans in question["answers"]],
            }  # question["answers"]
            ele["id"] = question["id"]
            extracted_data.append(ele)
    return pd.DataFrame(extracted_data)

In [4]:
test_json = 'covid-qa/covid-qa-test.json'
dev_json = 'covid-qa/covid-qa-dev.json'
train_json = 'covid-qa/covid-qa-train.json'

test_df = extract_data_fromJSON(test_json)
dev_df = extract_data_fromJSON(dev_json)
train_df = extract_data_fromJSON(train_json)

In [5]:
print(train_df.shape, dev_df.shape, test_df.shape)

(1417, 5) (203, 5) (375, 5)


In [6]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [7]:
dev_responses = {}
for i in range(dev_df.shape[0]):
    question = dev_df['question'][i]
    context = dev_df['context'][i]
    QA_input = {
        'question': question,
        'context': context
      }
    res = nlp(QA_input)
    dev_responses[test_df['id']] = res['answer']


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/question_answering.py:296: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  p_mask = np.asarray(


TypeError: unhashable type: 'Series'

In [ ]:
test_responses = {}
for i in range(test_df.shape[0]):
    question = test_df['question'][i]
    context = test_df['context'][i]
    QA_input = {
        'question': question,
        'context': context
      }
    res = nlp(QA_input)
    test_responses[test_df['id']] = res['answer']


In [ ]:
with open('dev_pred.json', 'w') as f:
    json.dump(dev_responses, f)
    
with open('test_pred.json', 'w') as f:
    json.dump(test_responses, f)

In [54]:
# dev_responses = pd.read_csv('dev_predictons.csv')
# test_responses = pd.read_csv('test_predictions.csv')

In [55]:
# dev_responses['answers'] = dev_responses.apply(lambda row: ast.literal_eval(row.predictions)['answer'], axis = 1)
# test_responses['answers'] = test_responses.apply(lambda row: ast.literal_eval(row.predictions)['answer'], axis = 1)

In [56]:
# dev_dict = {}
# for i in range(dev_responses.shape[0]):
#     dev_dict[int(dev_responses['id'][i])] = dev_responses['answers'][i]
    
# test_dict = {}
# for i in range(test_responses.shape[0]):
#     test_dict[int(test_responses['id'][i])] = test_responses['answers'][i]
    

In [57]:
# with open('dev_pred.json', 'w') as f:
#     json.dump(dev_dict, f)
    
# with open('test_pred.json', 'w') as f:
#     json.dump(test_dict, f)


In [58]:
! python evaluate.py covid-qa/covid-qa-dev.json dev_pred.json --out-file dev_results.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [59]:
! python evaluate.py covid-qa/covid-qa-test.json test_pred.json --out-file test_results.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
